In [1]:
import os
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table, join, hstack
import astropy.units as u

import scipy.stats as stats

plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 24,
    'axes.labelsize': 'large',
    'mathtext.fontset': 'stix'
})

def errJy_to_errABmag(flux_Jy, fluxerr_Jy):
    return -2.5 / (flux_Jy * np.log(10)) * fluxerr_Jy

## GAMA catalog details

In [2]:
#download GAMA from the database
with fits.open('http://www.gama-survey.org/dr4/data/cat/gkvInputCat/v02/gkvInputCatv02.fits') as cat:
    gkvInputCat = Table(cat[1].data)

with fits.open('http://www.gama-survey.org/dr4/data/cat/StellarMasses/v24/StellarMassesGKVv24.fits') as cat:
    StellarMassesGKV = Table(cat[1].data)

GAMA_mstar_concat = join(gkvInputCat, StellarMassesGKV)

KeyboardInterrupt: 

In [ ]:
#OR, if the joined catalog exists, import GAMA:
with fits.open('/data2/lsajkov/mpdg/data/GAMA/GAMA_mstars_concat.fits') as hdu:
    GAMA_mstar_concat = Table(hdu[1].data)

Redshifts

In [ ]:
#redshift distribution
redshifts = GAMA_mstar_concat[~np.isnan(GAMA_mstar_concat['Z']) &\
                                       (GAMA_mstar_concat['Z'] >= 0)]['Z']

print('= GAMA redshifts =')
print('counting non-nan ')
print('and >= 0 only')
print(f'total:\t{len(redshifts)}')
print(f'min:\t{np.min(redshifts):.3f}')
print(f'max:\t{np.max(redshifts):.3f}')
print(f'mean:\t{np.mean(redshifts):.3f}')
print(f'median:\t{np.median(redshifts):.3f}')
print(f'std:\t{np.std(redshifts):.3f}')
print('==================')

In [ ]:
fig = plt.figure(figsize = (10, 8))
ax  = fig.add_subplot()

GAMA_redshift_subset = redshifts[(redshifts >= 0.) &\
                       (redshifts < 2)]

ax.hist(GAMA_redshift_subset,
        bins = 75, histtype = 'step',
        linewidth = 3, color = 'black', density = True)

ax.set_xlabel('Redshift')
ax.text(0.45, 0.75,
        f'Limited to $0 < z < 2$\n\nIn this sample:\n{len(GAMA_redshift_subset)}/{len(GAMA_redshifts)} (~{int(100 * len(GAMA_redshift_subset)/len(GAMA_redshifts))} %)',
        transform = ax.transAxes,
        ma = 'right')

ax.axvspan(0.025, 0.5, color = 'lightgreen', alpha = 0.25)

Magnitude distribution

In [ ]:
g_snr = GAMA_mstar_concat['flux_gt']/GAMA_mstar_concat['flux_err_gt']
g_mag = GAMA_mstar_concat['flux_gt'] * u.Jy
g_mag = g_mag.to(u.ABmag).value

r_snr = GAMA_mstar_concat['flux_rt']/GAMA_mstar_concat['flux_err_rt']
r_mag = GAMA_mstar_concat['flux_rt'] * u.Jy
r_mag = r_mag.to(u.ABmag).value

In [ ]:
fig = plt.figure(figsize = (12, 10))
ax  = fig.add_subplot()

snr_thresh = 100

bins = ax.hist(g_mag[(g_mag > 0) & (g_snr > snr_thresh)],
        bins = 'auto',
        histtype = 'step', color = 'green',
        linewidth = 3)
ax.hist(r_mag[(r_mag > 0) & (r_snr > snr_thresh)],
        bins = bins[1], 
        histtype = 'step', color = 'red',
        linewidth = 3)

ax.text(0.01, 0.9,
        f'r- and g-band magnitudes in GAMA\nApplied cuts: SNR > {snr_thresh} & mag > 0',
        transform = ax.transAxes)

ax.set_xlabel('Magnitude ($r$ or $g$)')

R-band effective fwhm radius

In [ ]:
#the approximate area defined by the fwhm is 76%
#the area within R50 is 50%
#therefore, to scale R50 up to Rfwhm, we must scale up by
r50_to_fwhm_scale_factor = np.sqrt(0.76/0.50)

In [ ]:
r50_to_fwhm_scale_factor

In [ ]:
def gaussian(x, mean, std):
    return 1 / (np.sqrt(2 * np.pi * std**2)) * np.exp(-(x - mean) ** 2/(2 * std ** 2))

In [ ]:
std = 1

xs, ys = np.meshgrid(np.linspace(-3, 3, 501),
                    np.linspace(-3, 3, 501))
gauss2d = gaussian(xs, 0, std) * gaussian(ys, 0, std)
gauss2d /= np.sum(gauss2d)

fwhm_idx  = np.abs(gauss2d - np.max(gauss2d)/2) < 1e-3
fwhm_ring = np.where(fwhm_idx, True, np.nan)
fwhm_radius_x = (np.max(xs[fwhm_idx]) - np.min(xs[fwhm_idx])) / 2
fwhm_radius_y = (np.max(ys[fwhm_idx]) - np.min(ys[fwhm_idx])) / 2
fwhm_area = np.where(gauss2d > np.max(gauss2d)/2,
                    True, np.nan) 

thresh = np.max(gauss2d) * 0.95
while np.sum(gauss2d[gauss2d > thresh]) < 0.5:
    thresh -= 1e-7
half_light_idx  = gauss2d > thresh
half_light_radius_x = (np.max(xs[half_light_idx]) - np.min(xs[half_light_idx])) / 2
half_light_radius_y = (np.max(ys[half_light_idx]) - np.min(ys[half_light_idx])) / 2
half_light_ring = np.where(gauss2d > thresh, True, np.nan)

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax  = fig.add_subplot()

ax.imshow(gauss2d, origin = 'lower', cmap = 'Spectral')

In [ ]:
fwhm_radius_x/r50_to_fwhm_scale_factor

In [ ]:
half_light_radius_x

In [ ]:
half_light_radii = []
fwhm_radii = []

for std in np.linspace(0.5, 1.5, 15):
    xs, ys = np.meshgrid(np.linspace(-30, 30, 501),
                        np.linspace(-30, 30, 501))
    gauss2d = gaussian(xs, 0, std) * gaussian(ys, 0, std)
    gauss2d /= np.sum(gauss2d)

    fwhm_idx  = np.abs(gauss2d - np.max(gauss2d)/2) < 1e-3
    # fwhm_ring = np.where(fwhm_idx, True, np.nan)
    fwhm_radius_x = (np.max(xs[fwhm_idx]) - np.min(xs[fwhm_idx])) / 2
    # fwhm_radius_y = (np.max(ys[fwhm_idx]) - np.min(ys[fwhm_idx])) / 2
    # fwhm_area = np.where(gauss2d > np.max(gauss2d)/2,
    #                     True, np.nan) 

    thresh = np.max(gauss2d) * 0.95
    while np.sum(gauss2d[gauss2d > thresh]) < 0.5:
        thresh -= 1e-7
    half_light_idx  = gauss2d > thresh
    half_light_radius_x = (np.max(xs[half_light_idx]) - np.min(xs[half_light_idx])) / 2
    # half_light_radius_y = (np.max(ys[half_light_idx]) - np.min(ys[half_light_idx])) / 2
    # half_light_ring = np.where(gauss2d > thresh, True, np.nan)

    half_light_radii.append(half_lighthttp://desi.ibl.gov/_radius_x)
    fwhm_radii.append(fwhm_radius_x)
    
    plt.plot(gauss2d[250])

half_light_radii = np.array(half_light_radii)
fwhm_radii = np.array(fwhm_radii)

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax  = fig.add_subplot()

ax.imshow(gauss2d, origin = 'lower', cmap = 'Spectral')

# ax.imshow(fwhm_ring, origin = 'lower')

# thresh = np.max(gauss2d) * 0.95
# while np.sum(gauss2d[gauss2d > thresh]) < 0.5:
#     thresh -= 1e-7
# half_light_ring = np.where(gauss2d > thresh, True, np.nan)

# ax.imshow(half_light_ring, cmap = 'viridis', alpha = 0.3)
# ax.imshow(fwhm_area, cmap = 'viridis', alpha = 0.3)

In [ ]:
plt.plot(np.linspace(-2, 2, 100),
         gaussian(np.linspace(-2, 2, 100), 0, 0.5))

## GAMA Science Cat + Stellar Masses

In [ ]:
with fits.open('/data2/lsajkov/mpdg/data/GAMA/gkvScienceCatv02.fits') as cat:
    ScienceCat = Table(cat[1].data)

In [ ]:
#redshift distribution
redshifts = ScienceCat[~np.isnan(ScienceCat['Z']) &\
                                (ScienceCat['Z'] >= 0)]['Z']

print('= GAMA redshifts =')
print('counting non-nan ')
print('and >= 0 only')
print(f'total:\t{len(redshifts)}')
print(f'min:\t{np.min(redshifts):.3f}')
print(f'max:\t{np.max(redshifts):.3f}')
print(f'mean:\t{np.mean(redshifts):.3f}')
print(f'median:\t{np.median(redshifts):.3f}')
print(f'std:\t{np.std(redshifts):.3f}')
print('==================')

In [ ]:
fig = plt.figure(figsize = (10, 8))
ax  = fig.add_subplot()

redshift_idx = (redshifts >= 0.) & (redshifts < 1)
redshift_subset = redshifts[redshift_idx]


bins = ax.hist(redshift_subset,
        bins = 75, histtype = 'step',
        linewidth = 3, color = 'black', density = True)

highlight_idx = (redshifts >= bins[1][1]) & (redshifts <= 0.4)

ax.set_xlabel('Redshift')

plot_text = 'Limited to $0 < z < 1$\n\n' +\
           f'In this sample:\n{np.sum(redshift_idx)}/{len(redshift_idx)} (~{int(100 * np.sum(redshift_idx)/len(redshift_idx))} %)\n\n' +\
           f'In the highlighted area:\n{np.sum(highlight_idx)}/{len(highlight_idx)} (~{int(100 * np.sum(highlight_idx)/len(highlight_idx))} %)'

ax.text(0.45, 0.58,
        plot_text,
        transform = ax.transAxes,
        ma = 'right')

ax.axvspan(bins[1][1], 0.4, color = 'lightgreen', alpha = 0.25)

ax.text(bins[1][1] + 5e-3, 0.035, f'{bins[1][1]:.3f}', fontsize = 10, rotation = 90)
ax.text(0.4 - 15e-3, 0.035, 0.4, fontsize = 10, rotation = 90)

In [ ]:
r_mag = (ScienceCat['flux_rt'] * u.Jansky).to(u.ABmag).value
r_snr = ScienceCat['flux_rt']/ScienceCat['flux_err_rt']

g_mag = (ScienceCat['flux_gt'] * u.Jansky).to(u.ABmag).value
g_snr = ScienceCat['flux_gt']/ScienceCat['flux_err_gt']

u_mag = (ScienceCat['flux_ut'] * u.Jansky).to(u.ABmag).value
u_snr = ScienceCat['flux_ut']/ScienceCat['flux_err_ut']

In [ ]:
fig = plt.figure(figsize = (12, 10))
ax = fig.add_subplot()

cutoff = 400

bins = ax.hist(r_snr[r_snr < cutoff], bins = 'auto',
               histtype = 'step', color = 'red',
               linewidth = 3)[1]

ax.hist(g_snr[g_snr < cutoff],
        bins = bins,
        histtype = 'step', color = 'green', linewidth = 3)

ax.hist(u_snr[u_snr < cutoff],
        bins = bins,
        histtype = 'step', color = 'violet', linewidth = 3)

ax.set_xlabel('SNR\n(flux_xt/flux_err_xt)')
ax.set_ylabel('Count')

ax.axvline(5, color = 'black')

In [ ]:
fig = plt.figure(figsize = (12, 10))
ax = fig.add_subplot()

snr_thresh = 5
snr_mask = (u_snr > snr_thresh) &\
           (g_snr > snr_thresh) &\
           (r_snr > snr_thresh)

bins = ax.hist(r_mag[snr_mask],
               bins = 'auto', histtype = 'step',
               color = 'red', linewidth = 3)[1]

ax.hist(g_mag[snr_mask],
        bins = bins, histtype = 'step',
        color = 'green', linewidth = 3)

ax.hist(u_mag[snr_mask],
        bins = bins, histtype = 'step',
        color = 'violet', linewidth = 3)

ax.set_xlabel('Magnitude')
ax.set_ylabel('Count')

ax.set_xticks(np.arange(6.5, 21.5, 2))

In [ ]:
redshift_mask = (ScienceCat['Z'] >= 0.) & (ScienceCat['Z'] < 1)

snr_thresh = 5
snr_mask = (u_snr > snr_thresh) &\
           (g_snr > snr_thresh) &\
           (r_snr > snr_thresh)

total_mask = redshift_mask & snr_mask

## SOM-ready color catalog

In [2]:
with fits.open('/data2/lsajkov/mpdg/data/GAMA/gkvScienceCatv02.fits') as cat:
    ScienceCat = Table(cat[1].data)

with fits.open('/data2/lsajkov/mpdg/data/GAMA/StellarMassesGKVv24.fits') as cat:
    StellarMassesGKV = Table(cat[1].data)
    
joined_ScienceCat_StellarMassesGKV = join(ScienceCat, StellarMassesGKV)

In [4]:
#construct SOM catalog columns
cat = joined_ScienceCat_StellarMassesGKV

def errJy_to_errABmag(flux_Jy, fluxerr_Jy):
    return -2.5 / (flux_Jy * np.log(10)) * fluxerr_Jy

uberID = cat['uberID']

ra = cat['RAcen']
dec = cat['Deccen']

r_mag = (cat['flux_rt'] * u.Jansky).to(u.ABmag).value
r_mag_err = errJy_to_errABmag(cat['flux_rt'], cat['flux_err_rt'])
r_snr = cat['flux_rt']/cat['flux_err_rt']

g_mag = (cat['flux_gt'] * u.Jansky).to(u.ABmag).value
g_mag_err = errJy_to_errABmag(cat['flux_gt'], cat['flux_err_gt'])
g_snr = cat['flux_gt']/cat['flux_err_gt']

u_mag = (cat['flux_ut'] * u.Jansky).to(u.ABmag).value
u_mag_err = errJy_to_errABmag(cat['flux_ut'], cat['flux_err_ut'])
u_snr = cat['flux_ut']/cat['flux_err_ut']

i_mag = (cat['flux_it'] * u.Jansky).to(u.ABmag).value
i_mag_err = errJy_to_errABmag(cat['flux_it'], cat['flux_err_it'])
i_snr = cat['flux_it']/cat['flux_err_it']

Z_mag = (cat['flux_Zt'] * u.Jansky).to(u.ABmag).value
Z_mag_err = errJy_to_errABmag(cat['flux_Zt'], cat['flux_err_Zt'])
Z_snr = cat['flux_Zt']/cat['flux_err_Zt']

ug_color     = u_mag - g_mag
ug_color_err = np.sqrt(u_mag_err ** 2 + g_mag_err ** 2)

gr_color     = g_mag - r_mag
gr_color_err = np.sqrt(g_mag_err ** 2 + r_mag_err ** 2)

ri_color     = r_mag - i_mag
ri_color_err = np.sqrt(r_mag_err ** 2 + i_mag_err ** 2)

iZ_color     = i_mag - Z_mag
iZ_color_err = np.sqrt(i_mag_err ** 2 + Z_mag_err ** 2)

mstar     = cat['mstar']
mstar_err = cat['delmstar']

R50 = cat['R50']

redshift = cat['Z']
ppp      = cat['ppp']

NQ = cat['NQ']

/data2/lsajkov/mpdg/conda/envs/mpdg/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))
/var/tmp/lsajkov/ipykernel_3434293/3223120732.py:18: RuntimeWarning: divide by zero encountered in divide
  g_snr = cat['flux_gt']/cat['flux_err_gt']
/var/tmp/lsajkov/ipykernel_3434293/3223120732.py:22: RuntimeWarning: divide by zero encountered in divide
  u_snr = cat['flux_ut']/cat['flux_err_ut']
/var/tmp/lsajkov/ipykernel_3434293/3223120732.py:26: RuntimeWarning: divide by zero encountered in divide
  i_snr = cat['flux_it']/cat['flux_err_it']


In [5]:
#quality cuts
# ppp_cut = (ppp - np.mean(ppp[ppp > 0.05]))/np.std(ppp[ppp > 0.05]) < 2

snr_thresh = 5
snr_cut = (r_snr >= snr_thresh) &\
          (g_snr >= snr_thresh) &\
          (u_snr >= snr_thresh) &\
          (i_snr >= snr_thresh) &\
          (Z_snr >= snr_thresh)

redshift_cut = (redshift >= 0.002) & (redshift < 1)

NQ_cut = NQ >= 3

color_lo = -1
color_hi = 2
color_cut = (ug_color > color_lo) & (ug_color < color_hi) &\
            (gr_color > color_lo) & (gr_color < color_hi) &\
            (ri_color > color_lo) & (ri_color < color_hi) &\
            (iZ_color > color_lo) & (iZ_color < color_hi)

log_mstar_lo = 6
log_mstar_hi = 12
log_mstar_cut = (np.log10(mstar) > log_mstar_lo) & (np.log10(mstar) < log_mstar_hi)

R50_cut = R50 < 10

/var/tmp/lsajkov/ipykernel_3434293/2605397885.py:24: RuntimeWarning: invalid value encountered in log10
  log_mstar_cut = (np.log10(mstar) > log_mstar_lo) & (np.log10(mstar) < log_mstar_hi)


In [6]:
#define primary targeting region
def gr_color_condition(gr_color, gr_color_err,
               r_mag):
    
    return (gr_color - gr_color_err + 0.06 * (r_mag - 14)) < 0.9


def surf_bright_condition(surf_bright_r, surf_bright_r_err,
                                 r_mag):
    
    return (surf_bright_r + surf_bright_r_err\
            - 0.7 * (r_mag - 14)) > 18.5

In [7]:
prt_gr_color_cut = gr_color_condition(gr_color,
                                      gr_color_err,
                                      r_mag)

prt_mag_cut = r_mag > 18

In [8]:
total_cut = snr_cut &\
            NQ_cut &\
            redshift_cut &\
            color_cut &\
            log_mstar_cut &\
            R50_cut &\
            prt_gr_color_cut &\
            prt_mag_cut

In [9]:
print('Grand total of cuts:')
print(f'{np.sum(total_cut)}/{len(total_cut)} (~{int(100 * np.sum(total_cut)/len(total_cut))}%)')

Grand total of cuts:
48620/361748 (~13%)


In [11]:
GAMA_SOM_training_catalog = Table([uberID[total_cut],
                                   ra[total_cut], dec[total_cut],
                                   r_mag[total_cut],    r_mag_err[total_cut],
                                   g_mag[total_cut],    g_mag_err[total_cut],
                                   u_mag[total_cut],    u_mag_err[total_cut],
                                   i_mag[total_cut],    i_mag_err[total_cut],
                                   Z_mag[total_cut],    Z_mag_err[total_cut],
                                   ug_color[total_cut], ug_color_err[total_cut],
                                   gr_color[total_cut], gr_color_err[total_cut],
                                   ri_color[total_cut], ri_color_err[total_cut],
                                   iZ_color[total_cut], iZ_color_err[total_cut],                                    
                                   mstar[total_cut],    mstar_err[total_cut],
                                   redshift[total_cut], 
                                   # ppp[total_cut],
                                   R50[total_cut]],
                           names = ['ID',
                                    'ra', 'dec',
                                    'r_mag',    'r_mag_err',
                                    'g_mag',    'g_mag_err',
                                    'u_mag',    'u_mag_err',
                                    'i_mag',    'i_mag_err',
                                    'z_mag',    'z_mag_err',                                    
                                    'ug_col',   'ug_col_err', 
                                    'gr_col',   'gr_col_err',
                                    'ri_col',   'ri_col_err',
                                    'iz_col',   'iz_col_err',
                                    'mstar',    'mstar_err',
                                    'redshift',
                                    # 'ppp',
                                    'half_light_radius'])

In [12]:
GAMA_SOM_training_catalog.write('/data2/lsajkov/mpdg/data_products/GAMA/GAMA_SOM_training_catalog_18Jul24.fits',
                                overwrite = False)

### Log of training catalog parameters:

**July 4, 2024**:<br>
$\mathrm{SNR} > 5$,<br>
$0<z<1$,<br>
$-3<color<3$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

**July 5, 2024**:<br>
$\mathrm{SNR} > 5$,<br>
$0<z<1$,<br>
$-3<color<3$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

**July 8, 2024**:<br>
*Using observed magnitudes, rather than restframe* <br>
$\mathrm{SNR} > 5$,<br>
$0<z<1$,<br>
$-3<color<3$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

**July 12, 2024**:<br>
*Using observed magnitudes, rather than restframe* <br>
*Added $Z$-band data* <br>
*reorganized colors* <br>
$\mathrm{SNR} > 5$,<br>
$0<z<1$,<br>
$-1<color<2$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

**July 15, 2024**:<br>
*Using observed magnitudes, rather than restframe* <br>
*Added $Z$-band data* <br>
*reorganized colors* <br>
*added posterior predictive p-value (ppp) cut* <br>
$\mathrm{SNR} > 5$,<br>
$0<z<1$,<br>
$-1<color<2$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
$\frac{ppp - \mu(ppp)}{\sigma(ppp)} < 2$ <br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

**July 17, 2024**:<br>
*Using observed magnitudes, rather than restframe* <br>
*Added $Z$-band data* <br>
*reorganized colors* <br>
*added NQ flag cut* <br>
$\mathrm{SNR} > 5$,<br>
$0.002<z<1$,<br>
$\texttt{NQ} \ge 3$ <br>
$-1<color<2$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
$\frac{ppp - \mu(ppp)}{\sigma(ppp)} < 2$ <br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

**July 17, 2024**:<br>
*Using observed magnitudes, rather than restframe* <br>
*Added $Z$-band data* <br>
*reorganized colors* <br>
*added NQ flag cut* <br>
*added RA/Dec data* <br>
$\mathrm{SNR} > 5$,<br>
$0.002<z<1$,<br>
$\texttt{NQ} \ge 3$ <br>
$-1<color<2$,<br>
$6<\log(M_*)<12$,<br>
$R_{50}<10$<br>
$\frac{ppp - \mu(ppp)}{\sigma(ppp)} < 2$ <br>
\+ Everything is within the primary targeting region as defined **only** in Thornton+23 condition (1)

In [ ]:
fig = plt.figure(figsize = (12, 10))
ax = fig.add_subplot()

ax.hist(GAMA_SOM_training_catalog['redshift'], bins = 75,
        histtype = 'step', color = 'black', linewidth = 3)

ax.set_xlabel('Redshift')
ax.set_ylabel('Count')

fig.savefig('/data2/lsajkov/mpdg/figures/primary_targeting_region/GAMA_ptr_redshift_08jul2024.png',
            bbox_inches = 'tight')

In [ ]:
fig = plt.figure(figsize = (12, 10))
ax = fig.add_subplot()

ax.hist(GAMA_SOM_training_catalog['log_mstar'], bins = 75,
        histtype = 'step', color = 'black', linewidth = 3)

ax.set_xlabel('$\log(M_*/M_{\odot})$')
ax.set_ylabel('Count')

fig.savefig('/data2/lsajkov/mpdg/figures/primary_targeting_region/GAMA_ptr_logmstar_08jul2024.png',
            bbox_inches = 'tight')

In [ ]:
fig = plt.figure(figsize = (12, 10), constrained_layout = True)
ax  = fig.add_subplot()

plot_cut = snr_cut &\
           redshift_cut &\
           color_cut &\
           log_mstar_cut &\
           R50_cut &\
           prt_gr_color_cut

hb  = ax.hexbin(r_mag[plot_cut],
                gr_color[plot_cut],
                log_mstar[plot_cut],
                reduce_C_function = np.median,
                cmap = 'inferno',
                vmin = 6, vmax = 12)

plot_text = 'GAMA dataset\n(gkvScienceCatv02 +\nStellarMassesGKVv24)\n\n' +\
            'Primary targeting region\n' +\
           f'Accepted: {np.sum(total_cut)}/{len(total_cut)} (~{int(100 * np.sum(total_cut)/len(total_cut))}%)'

ax.text(0.025, 0.025, plot_text, transform = ax.transAxes)

ax.set_xlabel('$r$')
ax.set_ylabel('$g_{\mathrm{abs}}-r_{\mathrm{abs}}$')

ax.axvspan(18, 21, color = 'blue', alpha = 0.15, zorder = 0)

ax.set_xlim(7.5, 20.8)

fig.colorbar(mappable = hb, label = 'Median log$(M_*/M_{\odot})$')

fig.savefig('/data2/lsajkov/mpdg/figures/primary_targeting_region/GAMA_primary_targ_reg_gr_col_08jul2024.png',
            dpi = 150, bbox_inches = 'tight')

## SOM-ready panchromatic catalog

In [3]:
with fits.open('/data2/lsajkov/mpdg/data/GAMA/gkvScienceCatv02.fits') as cat:
    ScienceCat = Table(cat[1].data)

with fits.open('/data2/lsajkov/mpdg/data/GAMA/StellarMassesGKVv24.fits') as cat:
    StellarMassesGKV = Table(cat[1].data)
    
joined_ScienceCat_StellarMassesGKV = join(ScienceCat, StellarMassesGKV)

In [4]:
#construct SOM catalog columns
cat = joined_ScienceCat_StellarMassesGKV

def errJy_to_errABmag(flux_Jy, fluxerr_Jy):
    return -2.5 / (flux_Jy * np.log(10)) * fluxerr_Jy

photometric_bands = ['u', 'g', 'r', 'i', 'Z', 'Y', 'J', 'H', 'K']

uberID = cat['uberID']

ra = cat['RAcen']
dec = cat['Deccen']

magnitudes = Table()
SNRs = Table()

for band in photometric_bands:

    magnitudes.add_column((cat[f'flux_{band}t'] * u.Jansky).to(u.ABmag).value,
                          name = f'{band}_mag')
    magnitudes.add_column(errJy_to_errABmag(cat[f'flux_{band}t'], cat[f'flux_err_{band}t']),
                          name = f'{band}_mag_err')
    
    SNRs.add_column(cat[f'flux_{band}t']/cat[f'flux_err_{band}t'],
                    name = band)

colors = Table()
for i, band_hi in enumerate(photometric_bands):
    for band_lo in photometric_bands[i + 1:]:
        color_column = magnitudes[f'{band_hi}_mag'] - magnitudes[f'{band_lo}_mag']
        color_err_column = np.sqrt(magnitudes[f'{band_hi}_mag_err'] ** 2 +\
                                   magnitudes[f'{band_lo}_mag_err'] ** 2)
        colors.add_columns([color_column, color_err_column],
                   names = [f'{band_hi}{band_lo}_col', f'{band_hi}{band_lo}_col_err'])

mstar     = cat['mstar']
mstar_err = cat['delmstar']

R50 = cat['R50']

redshift = cat['Z']
ppp      = cat['ppp']

NQ = cat['NQ']

/data2/lsajkov/mpdg/conda/envs/mpdg/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))
/var/tmp/lsajkov/ipykernel_3945005/776854917.py:24: RuntimeWarning: divide by zero encountered in divide
  SNRs.add_column(cat[f'flux_{band}t']/cat[f'flux_err_{band}t'],


In [9]:
#quality_cuts

SNR_thresh = 5
SNR_mask = SNRs['r'] > SNR_thresh
# SNR_mask = np.ones(len(cat), dtype = bool)
# for band in photometric_bands:
#     SNR_mask = SNR_mask & (SNRs[band] > SNR_thresh)

redshift_mask = (redshift >= 0.002) & (redshift < 1)

NQ_mask = NQ >= 3

color_lo = -1
color_hi = 3
color_mask = np.ones(len(cat), dtype = bool)
for color in colors.colnames:
    if ~color.endswith('_err'):
        color_mask = color_mask &\
                    (colors[color] > color_lo) & (colors[color] < color_hi) &\
                    ~np.isnan(colors[color])
        
log_mstar_lo = 6
log_mstar_hi = 12
log_mstar_mask = (np.log10(mstar) > log_mstar_lo) & (np.log10(mstar) < log_mstar_hi)

R50_mask = R50 < 10

r_mag_mask = magnitudes['r_mag'] > 18

/var/tmp/lsajkov/ipykernel_3945005/2162885304.py:17: DeprecationWarning: Bitwise inversion '~' on bool is deprecated. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~color.endswith('_err'):
/var/tmp/lsajkov/ipykernel_3945005/2162885304.py:24: RuntimeWarning: invalid value encountered in log10
  log_mstar_mask = (np.log10(mstar) > log_mstar_lo) & (np.log10(mstar) < log_mstar_hi)


In [10]:
total_mask = SNR_mask &\
             redshift_mask &\
             color_mask &\
             log_mstar_mask &\
             R50_mask &\
             r_mag_mask

In [11]:
GAMA_SOM_training_catalog = Table([uberID,
                                   ra, dec,
                                   mstar, redshift,
                                   redshift,
                                   R50],
                          names = ['ID',
                                   'ra', 'dec',
                                   'mstar', 'mstar_err',
                                   'redshift',
                                   'half_light_radius'])

GAMA_SOM_training_catalog.add_columns(colors.columns,
                                      indexes = [3] * len(colors.colnames))

GAMA_SOM_training_catalog.add_columns(magnitudes.columns,
                                      indexes = [3] * len(magnitudes.colnames))

GAMA_SOM_training_catalog = GAMA_SOM_training_catalog[total_mask]

In [13]:
GAMA_SOM_training_catalog.write('/data2/lsajkov/mpdg/data_products/GAMA/GAMA_SOM_panchromatic_catalog_30Jul24.fits',
                                overwrite = False)

## SOM-ready restframe catalog

In [2]:
with fits.open('/data2/lsajkov/mpdg/data/GAMA/gkvScienceCatv02.fits') as cat:
    ScienceCat = Table(cat[1].data)

with fits.open('/data2/lsajkov/mpdg/data/GAMA/StellarMassesGKVv24.fits') as cat:
    StellarMassesGKV = Table(cat[1].data)
    
joined_ScienceCat_StellarMassesGKV = join(ScienceCat, StellarMassesGKV)

In [ ]:
#construct SOM catalog columns
cat = joined_ScienceCat_StellarMassesGKV

def errJy_to_errABmag(flux_Jy, fluxerr_Jy):
    return -2.5 / (flux_Jy * np.log(10)) * fluxerr_Jy

uberID = cat['uberID']

r_mag = (cat['flux_rt'] * u.Jansky).to(u.ABmag).value
r_mag_err = errJy_to_errABmag(cat['flux_rt'], cat['flux_err_rt'])
r_snr = cat['flux_rt']/cat['flux_err_rt']

g_mag = (cat['flux_gt'] * u.Jansky).to(u.ABmag).value
g_mag_err = errJy_to_errABmag(cat['flux_gt'], cat['flux_err_gt'])
g_snr = cat['flux_gt']/cat['flux_err_gt']

u_mag = (cat['flux_ut'] * u.Jansky).to(u.ABmag).value
u_mag_err = errJy_to_errABmag(cat['flux_ut'], cat['flux_err_ut'])
u_snr = cat['flux_ut']/cat['flux_err_ut']

i_mag = (cat['flux_it'] * u.Jansky).to(u.ABmag).value
i_mag_err = errJy_to_errABmag(cat['flux_it'], cat['flux_err_it'])
i_snr = cat['flux_it']/cat['flux_err_it']

Z_mag = (cat['flux_Zt'] * u.Jansky).to(u.ABmag).value
Z_mag_err = errJy_to_errABmag(cat['flux_Zt'], cat['flux_err_Zt'])
Z_snr = cat['flux_Zt']/cat['flux_err_Zt']

ur_rest     = cat['uminusr']
ur_rest_err = cat['deluminusr']

gi_rest     = cat['gminusi']
gi_rest_err = cat['delgminusi']



mstar     = cat['mstar']
mstar_err = cat['delmstar']

R50 = cat['R50']

redshift = cat['Z']
ppp      = cat['ppp']